# Sauvegarde des fichiers sur le S3

Le notebook stocke les données du FTP brutes dans le dossier [00_RawData](https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/00_RawData/?region=eu-west-3).

Ni les fichiers ni l\'arborescence de stockage ne sont modifiés.

- Connexion au S3
- Liste les fichiers à envoyer et calcule leur destination
- Uploade le fichier dans S3

In [1]:
import utils as u
import os

In [2]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

# Connect to S3

In [ ]:
# Import S3 connectors librairies

In [3]:
aws_separator = "/"

In [ ]:
!pip install git+git://github.com/thomaspernet/aws-python

In [ ]:
!pip install --upgrade git+git://github.com/thomaspernet/aws-python

In [4]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_athena import service_athena

In [5]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata')

# List files to send and compute each destination

In [8]:
# List files to send and compute each destination
import os
source_path= 'flux-2018-to-send'
root_dest_path=aws_separator.join(['INPI', 'TC_1' , '00_RawData', 'public' ,'IMR_Donnees_Saisies' ,'tc'])

list_files=[]
for r,d,files in os.walk(source_path):
    for i, file in enumerate(files):
          if file.endswith(('.csv')):
                
                source_full_path=os.path.join(r,file)
                
                dest_path = aws_separator.join([root_dest_path,r.replace("\\","/")])

                list_files.append((source_full_path,dest_path))

[('flux-2018-to-send\\2018\\12\\26\\0101\\482\\0101_482_20181226_085507_10_ets_supprime_EVT.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux-2018-to-send/2018/12/26/0101/482'),
 ('flux-2018-to-send\\2018\\12\\26\\0101\\482\\0101_482_20181226_085507_11_obs.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux-2018-to-send/2018/12/26/0101/482'),
 ('flux-2018-to-send\\2018\\12\\26\\0101\\482\\0101_482_20181226_085507_12_actes.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux-2018-to-send/2018/12/26/0101/482'),
 ('flux-2018-to-send\\2018\\12\\26\\0101\\482\\0101_482_20181226_085507_13_comptes_annuels.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux-2018-to-send/2018/12/26/0101/482'),
 ('flux-2018-to-send\\2018\\12\\26\\0101\\482\\0101_482_20181226_085507_1_PM.csv',
  'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux-2018-to-send/2018/12/26/0101/482')]

# Send files to s3

In [9]:
# Send files to s3

for (full_path,destination_in_s3) in list_files:
    filename=full_path.split('\\')[-1]
    key = aws_separator.join([destination_in_s3,filename])
    if not s3.key_exist(key):
        s3.upload_file(full_path, destination_in_s3)